In [ ]:
#####################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
#####################################################################################
# Simulation of 3D pose reconstructed by the 3D pose estimation method VNect
#####################################################################################

In [48]:
#########################################################################################################
# Visualize pose in 3D
#########################################################################################################

# from __future__ import division, print_function
from geoutils import radToDeg, xyz_to_angles
from vpython import *
from scipy.io import loadmat
import numpy as np
from pykalman import KalmanFilter
from geo import *

VID = 'PID05Task3'
# VID = 'PID23Task3'
VID = 'PID17Task3'
VID = 'PID20Task3'

STOP_AT_FRAMES = [1360, 1375, 1390]
STOP_AT_FRAMES = [1375, 1390]
STOP_AT_FRAMES = [1390]
STOP_AT_FRAMES = []

# # STOP_AT_FRAMES = [405] # deformed for PID20Task2
# STOP_AT_FRAMES = [250] # deformed for PID20Task3
STOP_AT_FRAMES = [420] # deformed for PID20Task3
# STOP_AT_FRAMES = [460] # deformed for PID17Task3
# STOP_AT_FRAMES = [2465] # deformed for PID05Task3

show_labels = True
show_labels = False

# smooth_joints_pos = True
smooth_joints_pos = False

FR = 100

#################################################
showRobot = True # simultaneously send commands to the robot
showRobot = False

angles_names = [
    "HeadPitch", "HeadYaw", 
    "LShoulderRoll", "LShoulderPitch", "LElbowRoll", "LElbowYaw",
    "RShoulderRoll", "RShoulderPitch", "RElbowRoll", "RElbowYaw", 
    "HipRoll", "HipPitch"
]
if showRobot:
    print ("Showing on virtual robot ...")
    from naoqi import ALProxy
    IP = "127.0.0.1"
    port = 45637
    motionProxy = ALProxy("ALMotion", IP, port)
    # Reset robot to neutral pose
    for an in angles_names:
        angle_reset = 0.
        if an == 'LShoulderPitch' or an == 'RShoulderPitch':
            angle_reset = angle_reset + np.pi/2
        motionProxy.setAngles(an, angle_reset, 1.)
        
    jointID = {     # Mapping: joint names -> position in array as extracted
        'Pelvis'    : 14,
        #'RHip'      : 1,   # not used
        #'LHip'      : 2,   # not used
        #'Spine'     : 7,   # not used
        'Neck'      : 1,
        'Nose'      : 16,
        'HeadTop'   : 0,  
        'LShoulder' : 5,
        'LElbow'    : 6,
        'LWrist'    : 7,
        'RShoulder' : 2,
        'RElbow'    : 3,
        'RWrist'    : 4
    }
#################################################

joints_used = [0,1,2,3,4,5,6,7, 8, 11, 14,15,16] # first 17, w/o legs

joints_names = ['head_top','neck','right_shoulder','right_elbow','right_wrist','left_shoulder','left_elbow','left_wrist',
                'right_hip','right_knee','right_ankle','left_hip','left_knee','left_ankle',
                'pelvis','spine','head',
                'right_hand','left_hand','right_toe','left_toe']

joint_connections = [
    (0, 16), (1, 15), (14, 15), (1, 16),       (8, 14), (11, 14),     # central, blue
    (1, 2), (2, 3), (3, 4), (4, 17), (8, 9), (9, 10), (10, 19),       # right, red
    (1, 5), (5, 6), (6, 7), (7, 18), (11, 12), (12, 13), (13, 20),   # left, green
]
joint_connections_colors = [color.blue]*6 + [color.red]*7 + [color.green]*7
joints_colors = [color.white]*len(joints_names)

if smooth_joints_pos:
    data = np.load('./../Dataset/VNectPose_smoothed/' + VID + '.npz')['d']
else:
    data = loadmat('./../Dataset/VNectPose/' + VID + '.mat')['Mt']
    data = np.reshape(data, (-1,21,3))
print ("Shape =" , np.shape(data))

# Setup scene
scene = canvas()
scene.width = 960
scene.height = 600
scene.title = "VNect 3D pose reconstruction"
scene.scale = 400.
# scene.fov = np.pi /3.
scene.up=vector(0.,-1.,0.)     # To flip scene horiznotally !!!
scene.forward=vector(0.,0.,1.)     # To rotate around y !!!
scene.center=vector(0.,-150.,0.) # To shift down a but

scene.lights = []
scene.ambient=color.gray(0.8)
# scene.center = vec(1.5,2,0)
# box(texture=textures.stucco)
# box(texture=textures.stones)

# Texture uderneath
y_plane_pos = 0
plane_size = 1000
# a = vertex( pos=vector(-plane_size, y_plane_pos, -plane_size) )
# b = vertex( pos=vector(-plane_size, y_plane_pos,  plane_size) )
# c = vertex( pos=vector( plane_size, y_plane_pos,  plane_size) )
# d = vertex( pos=vector( plane_size, y_plane_pos, -plane_size) )
# quad(vs=[a,b,c,d], texture=textures.rug)

# SHOW PLANE UNDERNEATH THE FIGURE
#box(pos=vec(0,y_plane_pos,0), length=plane_size, height=1, width=plane_size, texture=textures.rough)

# SHOW AXES ARROWS
# ar_len = 300
# arrow(pos=vec(0,0,0), axis=vec(1,0,0), shaftwidth=10, length=ar_len) # Show axes
# arrow(pos=vec(0,0,0), axis=vec(0,1,0), shaftwidth=10, length=ar_len)
# arrow(pos=vec(0,0,0), axis=vec(0,0,1), shaftwidth=10, length=ar_len)

frame = 0
scene.caption = 'Frame: ' + str(frame)
Ps = []
Es = []
Ls = []
# Draw all joints requested
for i in joints_used:
    x = data[frame, i, 0]
    y = data[frame, i, 1]
    z = data[frame, i, 2]
    P = sphere(pos=vector(x,y,z), radius=10, color=joints_colors[i])#, #size_units="world")
    Ps.append(P)
    if show_labels:
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=20,
                yoffset=50, space=30,
                height=16, border=3,
                font='sans')
        Ls.append(L)

# Draw all links between joints
for i, (a,b) in enumerate(joint_connections):
    if a in joints_used and b in joints_used:
        ax = data[frame, a, 0]
        ay = data[frame, a, 1]
        az = data[frame, a, 2]
        bx = data[frame, b, 0]
        by = data[frame, b, 1]
        bz = data[frame, b, 2]     
        cx = (ax + bx) / 2.
        cy = (ay + by) / 2.
        cz = (az + bz) / 2.
        E = ellipsoid(pos=vector(cx,cy,cz), axis=vector(ax-bx,ay-by,az-bz), length=np.linalg.norm([ax-bx,ay-by,az-bz]), 
                  height=25, width=25, color=joint_connections_colors[i])
        Es.append(E)


# Animate
for frame in range(1, len(data)):
    
    ####################################################################
    # Calculate joint angles & Send commands to the robot
    if showRobot:
        xyz = data[frame]
        ####################################################################
        # Calculate 12 joint angles, if not possible for this method set NAN (np.nan)
        # xyz[jointID['Pelvis']] gives xyz coordinates of Pelvis joint

        jas = xyz_to_angles(Point(xyz[jointID['HeadTop']]), Point(xyz[jointID['Nose']]), 
                            Point(xyz[jointID['Neck']]),    Point(xyz[jointID['Pelvis']]), 
                Point(xyz[jointID['LShoulder']]), Point(xyz[jointID['LElbow']]), Point(xyz[jointID['LWrist']]), 
                Point(xyz[jointID['RShoulder']]), Point(xyz[jointID['RElbow']]), Point(xyz[jointID['RWrist']])                  
    )
        
        for ang_ind, ang in enumerate(jas):
            if not np.isnan(ang): # take only angles available for this method
                motionProxy.setAngles(angles_names[ang_ind], ang, 1.)
    
    rate(FR)
    scene.caption = 'Frame: ' + str(frame)
    # Draw all joints requested
    cnt = 0
    for i in joints_used:
        x = data[frame, i, 0]
        y = data[frame, i, 1]
        z = data[frame, i, 2]
        Ps[cnt].pos = vector(x,y,z)
        if show_labels:
            Ls[cnt].pos = vector(x,y,z)
        cnt += 1

    # Draw all links between joints
    cnt = 0
    for i, (a,b) in enumerate(joint_connections):
        if a in joints_used and b in joints_used:
            ax = data[frame, a, 0]
            ay = data[frame, a, 1]
            az = data[frame, a, 2]
            bx = data[frame, b, 0]
            by = data[frame, b, 1]
            bz = data[frame, b, 2]     
            cx = (ax + bx) / 2.
            cy = (ay + by) / 2.
            cz = (az + bz) / 2.
            Es[cnt].pos = vector(cx,cy,cz)
            Es[cnt].axis = vector(ax-bx,ay-by,az-bz)
            Es[cnt].length = np.linalg.norm([ax-bx,ay-by,az-bz])
            cnt += 1
            
#             ellipsoid(pos=vector(cx,cy,cz), axis=vector(ax-bx,ay-by,az-bz), length=np.linalg.norm([ax-bx,ay-by,az-bz]), 
#                       height=10, width=10, color=joint_connections_colors[i])
# text(text='My text is\ngreen', pos=vector(1,1,1), align='center', color=color.green)

    if frame in STOP_AT_FRAMES:
        break

Shape = (2946, 21, 3)


<IPython.core.display.Javascript object>